In [1]:
!pip install --no-cache-dir transformers sentencepiece

     |████████████████████████████████| 3.1 MB 5.2 MB/s 
     |████████████████████████████████| 1.2 MB 50.0 MB/s 
     |████████████████████████████████| 3.3 MB 42.8 MB/s 
     |████████████████████████████████| 61 kB 6.0 MB/s 
     |████████████████████████████████| 596 kB 47.8 MB/s 
     |████████████████████████████████| 895 kB 44.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
pip install datasets sacrebleu

     |████████████████████████████████| 298 kB 5.2 MB/s 
     |████████████████████████████████| 90 kB 8.8 MB/s 
     |████████████████████████████████| 132 kB 46.4 MB/s 
     |████████████████████████████████| 1.1 MB 37.6 MB/s 
     |████████████████████████████████| 243 kB 47.3 MB/s 
     |████████████████████████████████| 271 kB 49.9 MB/s 
     |████████████████████████████████| 160 kB 46.6 MB/s 
     |████████████████████████████████| 192 kB 44.1 MB/s 


We will see how to easily load the dataset for this task using 🤗 Datasets and how to fine-tune a model on it using the Trainer API.

In [3]:
model_checkpoint = "google/mt5-small"

In [4]:
from datasets import Dataset, load_metric, load_dataset

metric = load_metric("sacrebleu")

Downloading:   0%|          | 0.00/2.37k [00:00<?, ?B/s]

In [5]:
import torch
import numpy as np
import pandas as pd
import os
from google.colab import drive
import logging

In [6]:
drive.mount('content')

Mounted at content


In [7]:
PATH_TO_DATASET = "content/MyDrive"

In [8]:
data  =  pd.read_excel(os.path.join(PATH_TO_DATASET, "Data.xlsx"), index_col = 0)

In [9]:
data.head()

,Arabic_transcript,English_transcript
0,مرحبا بكم في مقدمة لعلوم البيانات مع بايثون. ه...,Welcome to an introduction to Data Science wit...
1,مرحباً، أنا (كريس بروكز)، هيئة التدريس هنا بكل...,"Hi, I'm Chris Brooks, faculty here at the Univ..."
2,مرحبا. أريد أن أريكم قليلا عن نظام دفتر جوبيتر...,Hi. I want to show you a little bit about the ...
3,في بقية هذه الوحدة، سأقوم بتقديم نظرة عامة أسا...,"In the rest of this module, I'm going to provi..."
5,تحدثنا عن السلاسل عندما تحدثنا عن القوائم والت...,We talked about strings when we talked about l...


In [10]:
data.shape

(16818, 2)

In [11]:
import random

random.seed(10)

train_set, validate_set, test_set = np.split(data.sample(frac=1), [int(.8*len(data)), int(.9*len(data))])
# https://datascience.stackexchange.com/questions/15135/train-test-validation-set-splitting-in-sklearn 



In [12]:
train_set.shape, validate_set.shape, test_set.shape

((13454, 2), (1682, 2), (1682, 2))

In [13]:
train_set.isnull().sum() / train_set.shape[0] *100

Arabic_transcript     0.037164
English_transcript    0.014865
dtype: float64

In [14]:
train_set.dropna(inplace=True)
validate_set.dropna(inplace=True)

In [15]:
train_set.isnull().sum() / train_set.shape[0] *100

Arabic_transcript     0.0
English_transcript    0.0
dtype: float64

In [16]:
train_set = train_set.drop_duplicates(subset=['Arabic_transcript', 'English_transcript'])

# train_set.head()
train_set['English_transcript'] = train_set.apply(lambda row: row.English_transcript.lower(), axis=1)
train_set.columns = ['ar', 'en']

In [17]:
validate_set = validate_set.drop_duplicates(subset=['Arabic_transcript', 'English_transcript'])

# train_set.head()
validate_set['English_transcript'] = validate_set.apply(lambda row: row.English_transcript.lower(), axis=1)
validate_set.columns = ['ar', 'en']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


text generated seems to be not random. I have got something wrong here

### Pre-processing for arabic text?

In [18]:
train_set.head()

,ar,en
2613,الآن ، اسمحوا لي أيضًا أن أعرّف المتجه y لأخذ كل,now let me also define a vector y to be taking...
1216,يحاولون جعله يعمل بشكل أسرع ، أليس كذلك؟,"they try to make it run faster, right?"
918,، سيكون الإدخال X متعدد الأبعاد.,the input x will be multi-dimensional.
6222,حسنًا ، أعتقد أنها طريقة مفيدة للتفكير في الأم...,"um, so- so- i think it is a useful way to thin..."
11595,الدورة الرابعة والخامسة على,course four and five are focusing on


**Text cleaning?**

In [19]:
pip install mpu

     |████████████████████████████████| 69 kB 3.6 MB/s 


In [20]:
train_set['translation'] = train_set.apply(lambda row: {'en':row['en'], 'ar':row['ar']},
                            axis=1)
train_set = train_set.drop(columns = ['ar', 'en'])  #https://stackoverflow.com/questions/55136065/convert-multiple-pandas-column-into-json 
train_set.head()

,translation
2613,{'en': 'now let me also define a vector y to b...
1216,"{'en': 'they try to make it run faster, right?..."
918,{'en': 'the input x will be multi-dimensional....
6222,"{'en': 'um, so- so- i think it is a useful way..."
11595,"{'en': 'course four and five are focusing on',..."


In [21]:
validate_set['translation'] = validate_set.apply(lambda row: {'en':row['en'], 'ar':row['ar']},
                            axis=1)
validate_set = validate_set.drop(columns = ['ar', 'en'])  #https://stackoverflow.com/questions/55136065/convert-multiple-pandas-column-into-json 
validate_set.head()

,translation
13675,{'en': 'do i have any questions about logistic...
11829,"{'en': 'so, this was a great project in the fi..."
15342,"{'en': 'you have a train set, and you have a t..."
16063,{'en': 'says keep on going this way and maybe ...
13506,"{'en': 'make decisions, observe outcomes and t..."


In [22]:
import json

import mpu.io
# https://www.py4u.net/discuss/191017 
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_json.html


with open('data.json', 'w', encoding='utf-8') as file:
    train_set.to_json(file, orient="records", force_ascii=False, lines = True)

translation = load_dataset('json', data_files= 'data.json')
# https://huggingface.co/docs/datasets/loading_datasets.html#json-files 

Using custom data configuration default-736fefb5dc4e0589


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-736fefb5dc4e0589/0.0.0/c2d554c3377ea79c7664b93dc65d0803b45e3279000f993c7bfd18937fd7f426. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [23]:
with open('data_val.json', 'w', encoding='utf-8') as file:
    validate_set.to_json(file, orient="records", force_ascii=False, lines = True)

translation_val = load_dataset('json', data_files= 'data_val.json')
# https://huggingface.co/docs/datasets/loading_datasets.html#json-files 

Using custom data configuration default-6263f2d1f58f7896


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-6263f2d1f58f7896/0.0.0/c2d554c3377ea79c7664b93dc65d0803b45e3279000f993c7bfd18937fd7f426. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [24]:
translation

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 13307
    })
})

In [25]:
translation_val

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 1671
    })
})

# Fine tuning 
I will be using 'data' dataset. It contains (2 coursera courses, springer file, DS_codata_org, yt_stanford). 

I will mainly depend on this Nb from huggingface notebooks
https://github.com/huggingface/notebooks/blob/master/examples/translation.ipynb 

In [26]:
cuda = torch.device('cuda') 

### Create custom Datasets Class

## Preprocessing the data

In [27]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=False)  #https://discuss.huggingface.co/t/error-with-new-tokenizers-urgent/2847/4

Downloading:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/553 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.11M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

In [28]:
if "t5" in model_checkpoint:
    tokenizer.src_lang = "en-XX"
    tokenizer.tgt_lang = "ar-AR"
    tokenizer.source_prefix = "translate English to Arabic: "

In [29]:
tokenizer("Hello, this one sentence!")

{'input_ids': [30273, 261, 714, 1371, 259, 98923, 309, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [30]:
tokenizer(["Hello, this one sentence!", "This is another sentence."])

{'input_ids': [[30273, 261, 714, 1371, 259, 98923, 309, 1], [1494, 339, 259, 7845, 259, 98923, 260, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1]]}

In [31]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "translate English to Romanian: "
else:
    prefix = ""

In [32]:
max_input_length = 128
max_target_length = 128
source_lang = "en"
target_lang = "ar"

def preprocess_function(examples):
    inputs = [prefix + ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [33]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [34]:
show_random_elements(translation["train"])

,translation
0,"{'en': '[noise] and will output the estimated [noise] price.', 'ar': '[ضوضاء] وستخرج القيمة المقدرة سعر [ضوضاء].'}"
1,"{'en': 'partial derivative of this term with respect to theta j is equal to xj, okay?', 'ar': 'والمشتق الجزئي لهذا الحد فيما يتعلق بـ Theta J يساوي XJ ، حسنًا؟'}"
2,"{'en': 'we're trying to discriminate between positive and negative classes.', 'ar': 'نحن نحاول التمييز بين الطبقات الإيجابية والسلبية.'}"
3,"{'en': 'and then future cost of that relaxed problem is just going to be a heuristic,', 'ar': 'ومن ثم فإن التكلفة المستقبلية لهذه المشكلة المريحة ستكون مجرد إرشاد ،'}"
4,"{'en': 'uh, i think, uh, this year,', 'ar': 'آه ، أعتقد ، آه ، هذا العام ،'}"


In [35]:
metric

Metric(name: "sacrebleu", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id='references')}, usage: """
Produces BLEU scores along with its sufficient statistics
from a source against one or more references.

Args:
    predictions: The system stream (a sequence of segments).
    references: A list of one or more reference streams (each a sequence of segments).
    smooth_method: The smoothing method to use. (Default: 'exp').
    smooth_value: The smoothing value. Only valid for 'floor' and 'add-k'. (Defaults: floor: 0.1, add-k: 1).
    tokenize: Tokenization method to use for BLEU. If not provided, defaults to 'zh' for Chinese, 'ja-mecab' for
        Japanese and '13a' (mteval) otherwise.
    lowercase: Lowercase the data. If True, enables case-insensitivity. (Default: False).
    force: Insist that your tokenized input is actually detokenized.

Returns:
    'score': BLEU score,
    'counts'

### Tokenization

In [36]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

To apply this function on all the pairs of sentences in our dataset, we just use the map method of our dataset object we created earlier. This will apply the function on all the elements of all the splits in dataset, so our training, validation and testing data will be preprocessed in one single command.

In [37]:
tokenized_datasets = translation.map(preprocess_function, batched=True)

  0%|          | 0/14 [00:00<?, ?ba/s]

In [38]:
tokenized_datasets_val = translation_val.map(preprocess_function, batched=True)

  0%|          | 0/2 [00:00<?, ?ba/s]

##Fine-tuning the model


In [39]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

In [50]:
batch_size = 4
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-{source_lang}-to-{target_lang}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
    # push_to_hub=True,
)

In [51]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

we have to do a bit of pre-processing to decode the predictions into texts:

In [52]:
import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [53]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets_val["train"], # this is validation, But I couldn't create a dataset that is composed of training and validation. 
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Using amp fp16 backend


In [54]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `MT5ForConditionalGeneration.forward` and have been ignored: translation.
***** Running training *****
  Num examples = 13307
  Num Epochs = 1
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 3327


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,0.000000,nan,0.021600,2.362100


/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
Saving model checkpoint to mt5-small-finetuned-en-to-ar/checkpoint-500
Configuration saved in mt5-small-finetuned-en-to-ar/checkpoint-500/config.json
Model weights saved in mt5-small-finetuned-en-to-ar/checkpoint-500/pytorch_model.bin
tokenizer config file saved in mt5-small-finetuned-en-to-ar/checkpoint-500/tokenizer_config.json
Special tokens file saved in mt5-small-finetuned-en-to-ar/checkpoint-500/special_tokens_map.json
Copy vocab file to m

TrainOutput(global_step=3327, training_loss=0.0, metrics={'train_runtime': 1756.0584, 'train_samples_per_second': 7.578, 'train_steps_per_second': 1.895, 'total_flos': 1121940750336000.0, 'train_loss': 0.0, 'epoch': 1.0})

### get number of sentences

In [ ]:
from nltk.tokenize import word_tokenize, sent_tokenize
import nltk

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
train['input_text_sent'] = train['input_text'].apply(sent_tokenize).tolist()

In [ ]:
train['input_text_sent']

0      [hello and welcome., as you probably know, dee...
1      [hello, and welcome back., in this week we're ...
2      [in this video, we'll go over logistic regress...
5      [in this video, i want to help you gain an int...
6      [in this video, i'll show you a slightly more ...
                             ...                        
114    [in the last video, you saw how the attention ...
115    [one of the most exciting developments in deep...
116    [let's jump in to talk about the self-attentio...
117    [let's jump in and learn about the multi head ...
118    [you've learned about self attention, you've l...
Name: input_text_sent, Length: 99, dtype: object

In [ ]:
import string
nsentences = train['input_text'].str.split('.').map(len).sum()
# nsentences = train['input_text'].count()

In [ ]:
nsentences

6454

In [ ]:
import string
nsentences = train['target_text'].str.split('.').map(len).sum()
# nsentences = train['input_text'].count()

In [ ]:
nsentences

6302

In [ ]:
train['input_text'].head(1)

0    hello and welcome. as you probably know, deep ...
Name: input_text, dtype: object

In [ ]:
val = train.sample(frac = 0.05)
train = train.drop(index = val.index).astype(str)

In [ ]:
val.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5 entries, 15 to 94
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   target_text      5 non-null      object
 1   input_text       5 non-null      object
 2   input_text_sent  5 non-null      object
dtypes: object(3)
memory usage: 160.0+ bytes


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 94 entries, 0 to 118
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   target_text      94 non-null     object
 1   input_text       94 non-null     object
 2   input_text_sent  94 non-null     object
dtypes: object(3)
memory usage: 2.9+ KB


## Training the model

## 1. mt5-small - 300M parameters (small xD)

51M parameters

###Sidenote on GPU memory usage
The amount of GPU memory required to train a Transformer model depends on many different factors (maximum sequence length, number of layers, number of attention heads, size of the hidden dimensions, size of the vocabulary, etc.). Out of these, the maximum sequence length of the model is one of the most significant.
Also, mT5 has a much larger vocabulary than T5 (~250,000 tokens to ~32,000 tokens), contributing to mT5 being quite punishing in terms of GPU memory required.

In [49]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)


train_df = train
eval_df = val

train_df["prefix"] = ""
eval_df["prefix"] = ""

model_args = T5Args()

#The maximum sequence length of 100
#  allows the model to work with reasonably long text (typically a few sentences) while also keeping the training time practical.
model_args.max_seq_length = 100
#Generally, larger batch sizes mean better GPU utilization, and therefore, shorter training times
model_args.train_batch_size = 4
model_args.eval_batch_size = 4
model_args.num_train_epochs = 10
model_args.scheduler = "cosine_schedule_with_warmup"
model_args.evaluate_during_training = True
model_args.evaluate_during_training_steps = 10000
model_args.learning_rate = 0.0003
model_args.optimizer = 'Adafactor'
model_args.use_multiprocessing = False
model_args.fp16 = False
model_args.save_steps = -1
model_args.save_eval_checkpoints = False
model_args.no_cache = True
model_args.reprocess_input_data = True
model_args.overwrite_output_dir = True
model_args.save_model_every_epoch = False
model_args.preprocess_inputs = False
model_args.use_early_stopping = True
model_args.num_return_sequences = 1
model_args.do_lower_case = True
model_args.output_dir = "mt5/"
model_args.best_model_dir = "mt5/best_model"

#If you are using wandb add: wandb.login(key="API KEY")
model_args.wandb_project = "Yoruba mT5"

model = T5Model("mt5", "google/mt5-small", args=model_args)

# Train the model
model.train_model(train_df, eval_data=eval_df)

NameError: ignored

1. 6k sentences; started 1:50 --8 mins

## 2. mt5-base

## 3. google/mt5-large

- https://huggingface.co/transformers/notebooks.html
- https://github.com/huggingface/transformers/issues/8704